# Labs Access Analysis: 01 Data Download

## Setup
import all the needed libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from openrouteservice import client
import time
import json
import os

In [2]:
orstoken = ""
with open('orstoken.txt', 'r') as f:
    orstoken = f.readline()
    orstoken = orstoken.rstrip()

## data cleaning

In [3]:
# Load data
labs = pd.read_csv('data/labs.csv')

show the first 5 lines

In [4]:
labs.head(5)

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes,URL 2
0,3D Incubator,Incubator-Accelerator,Barcelona,BCN,"Polígono Industrial de la, Avinguda Parc Logís...",8040.0,41.340036,2.123036,Manually added,https://www.incubator3d.org/,NaN,NaN
1,AKASHA Hub Barcelona,Hackerspace,Barcelona,BCN,"carrer de la verneda, 17 loc1",8018.0,41.411054,2.191146,Hackerspaces.org,https://wiki.hackerspaces.org/AKASHA_Hub_Barce...,NaN,NaN
2,Ateneu de Fabricació Ca L'Alier,Ateneu de Fabricació,Barcelona,BCN,"Carrer de Pere IV, 362",8019.0,41.410786,2.203873,Ajuntament.barcelona.cat,NaN,NaN,NaN
3,Ateneu de Fabricació Ciutat Meridiana,Ateneu de Fabricació,Barcelona,BCN,"Av. de Rasos de Peguera, 232",8033.0,41.460460,2.169872,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN
4,Ateneu de Fabricació Gràcia,Ateneu de Fabricació,Barcelona,BCN,Carrer Perill 8,8012.0,41.399364,2.161167,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN


### description of the fields of the dataset
|  field name | description | details   |
|---|---|---|
| Name | name of the lab | |
| Type | type of Fablab. | *Fab Lab, Service, Makerspace, DIYBio Lab, Ateneu de Fabricació, City Space, Library, FabCafè, Incubator/Accelerator. Foundation* | 
| Cluster | which cluster the lab belongs to (Barcelona or Milan) | *BCN, MI* |
| Address | The house number where the lab is located | |
| CAP | the zipcode of the address | | 
| Lat | the latitude expressed in decimal degree | | 
| Long | the longitude expressed in decimal degree | |
| Source | where the information was collected from | *fablabs.io, hackerspaces.org, makerspaces.make.co, ajuntament.barcelona.cat, sphere.diybio.org, manually added, ajuntament de Barcelona policy document* | 
| URL | the web address of the lab | | 
| Notes | some information notes.  |the only note is if the lab has been closed (*Closed*), otherwise nothing appears. | 

separation of active labs from closed ones

In [5]:
closed_labs = labs[labs.Notes.str.contains("losed")==True]

In [6]:
closed_labs.shape

(10, 12)

In [7]:
closed_labs

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes,URL 2
25,Fab Lab Tinkerers,Fab Lab,Castelldefels,BCN,"PMT-UPC, RDIT Building, C/Esteve Terradas, 1",8860.0,41.276201,1.988951,Fablabs.io,https://fablabs.io/labs/Tinkerers,Closed,NaN
27,FabLab Ventura,Fab Lab,Milan,MI,"Via Privata Giovanni Ventura, 20",20134.0,45.484208,9.245129,Fablabs.io,https://fablabs.io/labs/fablabventura,Closed,NaN
29,Frankestein Garage,Fab Lab,Milan,MI,NaN,NaN,NaN,NaN,"Fablabs.io, Hackerspaces.org",https://fablabs.io/labs/frankensteingarage,Closed,NaN
36,Ideas Bit Factory,Service,Milan,MI,"Via Enrico Cosenz, 44/4",20158.0,45.511205,9.166396,Manually added,https://www.ideas-bit-factory.it/,Closed,NaN
37,IED Maker Lab,University Lab,Barcelona,BCN,"Carrer de Biada, 6",8012.0,41.405113,2.154622,Manually added,https://www.ied.edu/envira/barcelona-maker-lab,Closed,NaN
38,kernelpanic,Hackerspace,Barcelona,BCN,NaN,NaN,NaN,NaN,Hackerspaces.org,https://wiki.hackerspaces.org/KernelPanic,Closed,NaN
46,MakerspaceBCN,Makerspace,Barcelona,BCN,NaN,NaN,NaN,NaN,Makerspaces.make.co,http://makerspacebcn.org/,Closed,NaN
51,Openspace,Hackerspace,Milan,MI,via Bardolino 31,20142.0,45.425561,9.141824,Hackerspaces.org,https://wiki.hackerspaces.org/Openspace,Closed,NaN
52,PechBlenda Lab,DIYBio Lab,Barcelona,BCN,NaN,NaN,NaN,NaN,Sphere.diybio.org,https://sphere.diybio.org/labs/pechblenda/,Closed,NaN
61,Ripolab Hacklab,Hackerspace,Barcelona,BCN,NaN,NaN,NaN,NaN,Manually added,https://www.ripolab.org/,Closed,NaN


In [8]:
valid_labs = labs[~labs.index.isin(closed_labs.index)]

In [9]:
print("The total of the valid labs is %s" % valid_labs.shape[0])

The total of the valid labs is 77


show the first 10 in the dataset

In [10]:
valid_labs.head(10)

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes,URL 2
0,3D Incubator,Incubator-Accelerator,Barcelona,BCN,"Polígono Industrial de la, Avinguda Parc Logís...",8040.0,41.340036,2.123036,Manually added,https://www.incubator3d.org/,NaN,NaN
1,AKASHA Hub Barcelona,Hackerspace,Barcelona,BCN,"carrer de la verneda, 17 loc1",8018.0,41.411054,2.191146,Hackerspaces.org,https://wiki.hackerspaces.org/AKASHA_Hub_Barce...,NaN,NaN
2,Ateneu de Fabricació Ca L'Alier,Ateneu de Fabricació,Barcelona,BCN,"Carrer de Pere IV, 362",8019.0,41.410786,2.203873,Ajuntament.barcelona.cat,NaN,NaN,NaN
3,Ateneu de Fabricació Ciutat Meridiana,Ateneu de Fabricació,Barcelona,BCN,"Av. de Rasos de Peguera, 232",8033.0,41.460460,2.169872,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN
4,Ateneu de Fabricació Gràcia,Ateneu de Fabricació,Barcelona,BCN,Carrer Perill 8,8012.0,41.399364,2.161167,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN
5,Ateneu de Fabricació La Fàbrica del Sol / Espa...,Ateneu de Fabricació / Barcelona Recircula – T...,Barcelona,BCN,"Passeig de Salvat Papasseit, 1",8003.0,41.382352,2.191718,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,https://bcnsostenible.cat/web/punt/espai-dinte...
6,Ateneu de Fabricació Les Corts,Ateneu de Fabricació,Barcelona,BCN,c/ Evarist Arnús 59 i c/ Comtes de Bell-Lloc 192,8014.0,41.383661,2.135464,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN
7,Ateneu de Fabricació Parc Tecnológic,Ateneu de Fabricació,Barcelona,BCN,"Carrer de Marie Curie, 8-14",8042.0,41.436965,2.168955,Ajuntament.barcelona.cat,https://ajuntament.barcelona.cat/ateneusdefabr...,NaN,NaN
8,ATTA33,Service,Barcelona,BCN,"Carrer de Tànger, 33",8018.0,41.400318,2.189540,Manually added,https://atta33.com/,NaN,NaN
9,Barcelona Mobility LAB,Fab Lab,Barcelona,BCN,Passatge del Torrent de l'Estadella 56,8030.0,41.432152,2.199793,Fablabs.io,NaN,NaN,NaN


save the dataset on the filesystem

In [11]:
valid_labs.to_csv('data' + os.sep + 'valid_labs.csv')

## Calculation of the isochrones for each lab
the calculation is based on OpenRouteService using 15 minutes walking distance as the maximum time.

convert the Lat and Long in float to manage the geospatial information

In [12]:
valid_labs = valid_labs.astype({"Lat": float, "Long": float})

creation of the request for calculation of the isochrones to OpenRouteService according to the [API](https://openrouteservice.org/dev/#/api-docs/v2/isochrones) syntax

900 seconds = 15 minutes (city)


In [13]:
body = {"locations":[],"range":[900],"attributes":["total_pop","area","reachfactor"],"range_type":"time","interval":900,"area_units":"m","location_type":"start"}
locations = {}

In [14]:
# check if the latitude and longitude for each lab is valid
for index, row in valid_labs.iterrows():
    if np.isnan(row["Lat"]) == False and np.isnan(row["Long"]) == False:
      locations[row["Name"]] = [row["Long"], row["Lat"]]

In [15]:
print("We have %s labs where the latitude and longitude is present" % str(len(locations)))

We have 77 labs where the latitude and longitude is present


organize the header for the API request

In [16]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': orstoken,
    'Content-Type': 'application/json; charset=utf-8'
}

start with the requests.<Br/>
Attention:<Br/>
it is necessary to limit the number of requests per second

In [17]:
body["locations"] = []
data = []
for k,i in enumerate(locations):
    print(i, locations[i])
    body["locations"].append(locations[i])
    # 15 minutes city, at walking distance
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/foot-walking', json=body, headers=headers)
    body["locations"] = []
    print(call.status_code, call.reason)
    lab_dict = call.json()
    lab_dict["name"] = i
    data.append(lab_dict)
    print("---")
    time.sleep(30)

3D Incubator [2.1230359460327093, 41.340035586268726]
200 OK
---
AKASHA Hub Barcelona [2.1911459401482185, 41.41105433376162]
200 OK
---
Ateneu de Fabricació Ca L'Alier [2.203873110704254, 41.41078570052905]
200 OK
---
Ateneu de Fabricació Ciutat Meridiana [2.1698723266571536, 41.460459824619406]
200 OK
---
Ateneu de Fabricació Gràcia [2.1611670113120494, 41.39936443324356]
200 OK
---
Ateneu de Fabricació La Fàbrica del Sol / Espai d'Intercanvi Barceloneta [2.191718180625283, 41.382352379582926]
200 OK
---
Ateneu de Fabricació Les Corts [2.135464167132671, 41.38366087714364]
200 OK
---
Ateneu de Fabricació Parc Tecnológic [2.1689552843279274, 41.436964660556576]
200 OK
---
ATTA33 [2.189540141998292, 41.40031808724662]
200 OK
---
Barcelona Mobility LAB [2.199793097820458, 41.43215238774583]
200 OK
---
BDN Lab [2.238114638501468, 41.44280114524807]
200 OK
---
Camins Makers [2.1113697963011724, 41.38878736307806]
200 OK
---
Cibernarium Poble Nou (Barcelona Activa) [2.194483373013783, 41.4

In [18]:
# Check the data, first lab
data[0]

{'type': 'FeatureCollection',
 'bbox': [2.109858, 41.333106, 2.136626, 41.345904],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 900.0,
    'center': [2.123046609113957, 41.34002295988895],
    'area': 1257375.85,
    'reachfactor': 0.2562,
    'total_pop': 25847.0},
   'geometry': {'coordinates': [[[2.109858, 41.336682],
      [2.111475, 41.334295],
      [2.111614, 41.334134],
      [2.111828, 41.334225],
      [2.117692, 41.334835],
      [2.117735, 41.334865],
      [2.117848, 41.335207],
      [2.122971, 41.338277],
      [2.123159, 41.338206],
      [2.123324, 41.338138],
      [2.123493, 41.338069],
      [2.123663, 41.337999],
      [2.125111, 41.337491],
      [2.127581, 41.333293],
      [2.127888, 41.333106],
      [2.131512, 41.334057],
      [2.13323, 41.334836],
      [2.133268, 41.334858],
      [2.13329, 41.334872],
      [2.133215, 41.335128],
      [2.133188, 41.335193],
      [2.132823, 41.336145],
      [2.131745, 41.338286],
   

store the data in the "data" folder as json file

In [19]:
with open("data" + os.sep + "isochrones.json", 'w') as outfile:
    json.dump(data, outfile)